# Project UPDATE for ViT Hybrid (ConvStems) Setup:

### Architecture
Run pre-training with contrastive learning (MoCo framework)
* Use backbone - `ViT Hybrid`
* **Perform SSL Pre-training** of backbone using contrastive learning (MoCo) -> augment medical imaging dataset (Chest XRs) (Pre-text task); create positive and negative pairs
  * Produce: `moco_vit_hybrid_encoder.pth`
* **Transfer Learning:** Fine-tune pre-trained ResNet for Pneumonia Chest XR classification
  * Produce: `finetuned_vit_hybrid_medical.pth`

### Pre-training Dataset: CheXpert
* Subset: Pneumonia classification only; smaller dataset (to accomodate class imbalance)

### Fine-tuning Dataset: NIH Pneumonia Dataset

###

In [1]:
# import libraries
import os
import sys
import argparse
from tqdm import tqdm

import numpy as np
import seaborn as sns
import pandas as pd
import math
from copy import deepcopy

%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display

import pickle
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import random_split, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Collab Needs

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define filepaths to required input (scripts, data) and outputs

In [3]:
# ----------------------------------------------------
# Inputs root
# ----------------------------------------------------
FP_ROOT="/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/"

# ----------------------------------------------------
# Dataset info
# ----------------------------------------------------
# Chexpert
DATASET_PATH_SPLIT = FP_ROOT + "Data/CheXpert_reduced_dataset_split_v3.zip"

# ChexPert
TRAIN_LABELS_CSV = FP_ROOT + "Data/1_final_project_updated_names_train_moco.csv"
TEST_LABELS_CSV = FP_ROOT + "Data/1_final_project_updated_names_test_moco.csv"

LINEAR_TRAIN_LABELS_CSV = FP_ROOT + "Data/2_final_project_updated_names_train_linear.csv"
LINEAR_TEST_LABELS_CSV = FP_ROOT + "Data/2_final_project_updated_names_test_linear.csv"

# ----------------------------------------------------
# Model SRC
# ----------------------------------------------------
SRC_ROOT = FP_ROOT + "src/"
TRAIN_SCRIPT = f"{SRC_ROOT}/train_moco.py"
MOCO_FOLDER = f"{SRC_ROOT}/moco"

# ----------------------------------------------------
# Outputs
# ----------------------------------------------------
ROOT_ARTIFACT_SAVE = FP_ROOT + "artifacts/"

In [4]:
# Add the project /src to the system path
if SRC_ROOT not in sys.path:
    sys.path.append(SRC_ROOT)
    print(f"Added {SRC_ROOT} to sys.path")

Added /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src/ to sys.path


In [5]:
print(sys.path)

['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/tmp/tmpi0b1067y', '/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src/']


## Unzip data

ChexPert

In [6]:
# Unzip the dataset (image) files to /tmp
DATA_DEST_UNZIPPED = "/tmp/CheXpert_dataset/"
os.makedirs(DATA_DEST_UNZIPPED, exist_ok=True)
!unzip "{DATASET_PATH_SPLIT}" -d {DATA_DEST_UNZIPPED} # need to use "" to accomdate space

Streaming output truncated to the last 5000 lines.
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/train/patient49868_study4_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/train/patient49869_study1_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/train/patient49869_study1_view2_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/train/patient49869_study2_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/train/patient49873_study1_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/train/patient49878_study3_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/train/patient49879_study1_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/train/patient49879_study2_view1_frontal.jpg  
  inflating: /tmp/CheXpert_da

In [5]:
# Update for the unzipped sub-name
DATA_DEST_UNZIPPED = "/tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/"

## 1) Run Pre-training - Contrastive Learning

In [6]:
# Run from the src directory
%cd "/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src"

/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src


In [13]:
# Hyperparams
batch_size = 64

In [14]:
! python moco/train_moco_vit_hybrid.py \
    --train_csv_path "$TRAIN_LABELS_CSV" \
    --test_csv_path "$TEST_LABELS_CSV" \
    --root_dir "$DATA_DEST_UNZIPPED" \
    --artifact_root "$ROOT_ARTIFACT_SAVE" \
    --batch_size "$batch_size" \
    --n_epochs 15 \
    --test_num_classes 2 \
    --linear_n_epochs 10 \
    --linear_train_csv_path "$LINEAR_TRAIN_LABELS_CSV" \
    --linear_test_csv_path "$LINEAR_TEST_LABELS_CSV" \
    --model_type "VIT_Hybrid" \
    --out_model_name "vit_hybrid_moco_encoder_v3.pth" \
    --embedding_dim 192

Created log file:  /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/artifacts/moco_training_log_20251204_024027.txt
[2025-12-04 02:40:27] Creating MoCo medical image Train DataLoader (Hybrid ViT)... : from /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/Data/1_final_project_updated_names_train_moco.csv
Creating MoCo DataLoader for train data...
CSV Path: /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/Data/1_final_project_updated_names_train_moco.csv
Image Root Dir: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/
Loading dataset from: /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/Data/1_final_project_updated_names_train_moco.csv
Training Dataset size: 49500 images
Data Sample q shape: torch.Size([3, 224, 224]), Sample k shape: torch.Size([3, 224, 224])
[2025-12-04 02:40:27] Using ViT Hybrid backbone for MoCo
[2025-12-04 02:40:27] Note: No option to use pretrained weights; ignoring if set
[2025-12-04 02:40:27] Starting MoCo ViT Hybrid training for 15 epochs...
Epoch 0/

## Test Backbone with Linear Classifier

In [15]:
# Run from the src directory
%cd "/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src"

# Hyperparams
batch_size = 128

/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src


In [16]:
! python moco/test_moco_vit_hybrid.py \
    --root_dir "$DATA_DEST_UNZIPPED" \
    --artifact_root "$ROOT_ARTIFACT_SAVE" \
    --test_num_classes 2 \
    --linear_n_epochs 10 \
    --linear_train_csv_path "$LINEAR_TRAIN_LABELS_CSV" \
    --linear_test_csv_path "$LINEAR_TEST_LABELS_CSV" \
    --model_checkpoint "vit_hybrid_moco_encoder_v3.pth" \
    --batch_size "$batch_size"

Created log file:  /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/artifacts/moco_backbone_testing_log_20251204_064801.txt
[2025-12-04 06:48:01] Starting MoCo backbone testing...
[2025-12-04 06:48:01] Loading pretrained MoCo model from /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/artifacts/vit_hybrid_moco_encoder_v3.pth...
[2025-12-04 06:48:01] Loaded model. Missing keys: ['queue', 'queue_ptr', 'patch_embed.proj.0.weight', 'patch_embed.proj.0.bias', 'patch_embed.proj.2.weight', 'patch_embed.proj.2.bias', 'patch_embed.proj.4.weight', 'patch_embed.proj.4.bias', 'pos_encoding.pos_embed', 'pos_encoding.cls_token', 'transformer.encoder.layers.0.self_attn.in_proj_weight', 'transformer.encoder.layers.0.self_attn.in_proj_bias', 'transformer.encoder.layers.0.self_attn.out_proj.weight', 'transformer.encoder.layers.0.self_attn.out_proj.bias', 'transformer.encoder.layers.0.linear1.weight', 'transformer.encoder.layers.0.linear1.bias', 'transformer.encoder.layers.0.linear2.weight', 'transform